### Training the model

Import libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from sales import data_holder, data_info, common, feature_engineering
from xgboost import XGBRegressor

Read data produced with feature engineering

In [4]:
data = pd.read_csv('../data/temp/processed_data.csv', delimiter=',')
data_holder.set_data(data)
data_info.preview()

Data shape:  (876141, 39)


,Unnamed: 0,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,...,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year,y_sku_warehouse_next_7days_sales,y_sku_warehouse_next_14days_sales,y_sku_warehouse_next_30days_sales
0,0,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1,...,0.0,0,0,0,0,0,0,0,0,0
1,1,2018-08-02,28322FYBBL,28322FYBB,28322,LASCONDES,0.0,13782.0,13782,1,...,0.0,0,0,0,0,0,0,0,0,0
2,2,2018-08-02,44422BLKXL,44422BLK,44422,LASCONDES,0.0,8739.0,8739,1,...,0.0,0,0,0,0,0,0,0,0,0
3,3,2018-08-03,12010ZOOALL,12010ZOO,12010,MALLSPORT,72588.0,50084.0,50084,1,...,0.0,0,0,0,0,0,0,0,0,0
4,4,2018-08-03,12011ZOOALL,12011ZOO,12011,LADEHESA,81820.0,56807.0,56807,1,...,0.0,0,0,0,0,0,0,0,0,0


In [8]:
data[data['uqantity'] == 0].shape

(0, 39)

In [18]:
df = data_holder.get_data()
df.tail()

,Unnamed: 0,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,...,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year,y_sku_warehouse_next_7days_sales,y_sku_warehouse_next_14days_sales,y_sku_warehouse_next_30days_sales
876136,876136,2023-02-08,59179FRBKALL,59179FRBK,59179,ECOMMERCE,4463.0,15966.0,15966,1,...,1.00,23,1,0,1,1,9,0,0,0
876137,876137,2023-02-08,83580BLKL,83580BLK,83580,ECOMMERCE,32175.0,91597.0,91597,1,...,1.00,25,3,2,0,1,2,0,0,0
876138,876138,2023-02-08,84701CNYL,84701CNY,84701,ECOMMERCE,58591.0,60252.0,60252,1,...,1.00,69,11,0,0,0,4,0,0,0
876139,876139,2023-02-08,85240BLKM,85240BLK,85240,ECOMMERCE,37794.0,104202.0,104202,1,...,1.14,272,44,3,1,2,11,0,0,0
876140,876140,2023-02-08,BK825000ALL,BK825000ALL,BK825000ALL,ECOMMERCE,10198.0,15966.0,15966,1,...,1.38,425,65,55,0,2,11,0,0,0


In [19]:
df.sort_values('date').head()

,Unnamed: 0,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,...,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year,y_sku_warehouse_next_7days_sales,y_sku_warehouse_next_14days_sales,y_sku_warehouse_next_30days_sales
0,0,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1,...,0.0,0,0,0,0,0,0,0,0,0
1,1,2018-08-02,28322FYBBL,28322FYBB,28322,LASCONDES,0.0,13782.0,13782,1,...,0.0,0,0,0,0,0,0,0,0,0
2,2,2018-08-02,44422BLKXL,44422BLK,44422,LASCONDES,0.0,8739.0,8739,1,...,0.0,0,0,0,0,0,0,0,0,0
3,3,2018-08-03,12010ZOOALL,12010ZOO,12010,MALLSPORT,72588.0,50084.0,50084,1,...,0.0,0,0,0,0,0,0,0,0,0
4,4,2018-08-03,12011ZOOALL,12011ZOO,12011,LADEHESA,81820.0,56807.0,56807,1,...,0.0,0,0,0,0,0,0,0,0,0


In [20]:
df.tail()

,Unnamed: 0,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,...,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year,y_sku_warehouse_next_7days_sales,y_sku_warehouse_next_14days_sales,y_sku_warehouse_next_30days_sales
876136,876136,2023-02-08,59179FRBKALL,59179FRBK,59179,ECOMMERCE,4463.0,15966.0,15966,1,...,1.00,23,1,0,1,1,9,0,0,0
876137,876137,2023-02-08,83580BLKL,83580BLK,83580,ECOMMERCE,32175.0,91597.0,91597,1,...,1.00,25,3,2,0,1,2,0,0,0
876138,876138,2023-02-08,84701CNYL,84701CNY,84701,ECOMMERCE,58591.0,60252.0,60252,1,...,1.00,69,11,0,0,0,4,0,0,0
876139,876139,2023-02-08,85240BLKM,85240BLK,85240,ECOMMERCE,37794.0,104202.0,104202,1,...,1.14,272,44,3,1,2,11,0,0,0
876140,876140,2023-02-08,BK825000ALL,BK825000ALL,BK825000ALL,ECOMMERCE,10198.0,15966.0,15966,1,...,1.38,425,65,55,0,2,11,0,0,0


Generate train and test datasets

In [21]:
cutoff_date = '2023-01-01'
train_data = df[df['date'] < cutoff_date]
test_data = df[df['date'] >= cutoff_date]

In [22]:
train_data.tail()

,Unnamed: 0,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,...,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year,y_sku_warehouse_next_7days_sales,y_sku_warehouse_next_14days_sales,y_sku_warehouse_next_30days_sales
848931,848931,2022-12-31,66090FLMP12M,66090FLMP,66090,ECOMMERCE,7064.0,24370.0,24370,1,...,1.0,4,1,0,0,3,4,0,0,0
848932,848932,2022-12-31,68311NENAS,68311NENA,68311,ECOMMERCE,34157.0,28739.0,28739,1,...,1.0,6,0,0,0,1,6,1,1,1
848933,848933,2022-12-31,72217TCWBXS,72217TCWB,72217,ECOMMERCE,11770.0,20168.0,20168,1,...,0.0,0,0,0,0,0,0,0,0,0
848934,848934,2022-12-31,84212MEORL,84212MEOR,84212,ECOMMERCE,42688.0,150420.0,150420,1,...,0.0,4,0,0,0,0,4,0,0,0
848935,848935,2022-12-31,85245CNYS,85245CNY,85245,ECOMMERCE,35727.0,83193.0,83193,1,...,1.0,15,0,0,1,5,15,0,0,1


In [23]:
df[(df['sku'] == '85245CNYS') &
   (df['warehouse'] == 'ECOMMERCE') &
   (df['month'] == 12)]

,Unnamed: 0,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,...,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year,y_sku_warehouse_next_7days_sales,y_sku_warehouse_next_14days_sales,y_sku_warehouse_next_30days_sales
832420,832420,2022-12-13,85245CNYS,85245CNY,85245,ECOMMERCE,33672.0,83193.0,83193,1,...,0.0,10,3,0,0,0,10,3,3,5
833238,833238,2022-12-14,85245CNYS,85245CNY,85245,ECOMMERCE,33672.0,83193.0,83193,1,...,1.0,11,1,0,1,1,11,2,2,4
834596,834596,2022-12-15,85245CNYS,85245CNY,85245,ECOMMERCE,33672.0,83193.0,83193,1,...,1.0,12,3,0,2,2,12,1,2,3
839484,839484,2022-12-19,85245CNYS,85245CNY,85245,ECOMMERCE,33672.0,83193.0,83193,1,...,1.0,13,1,0,0,3,13,0,2,3
848835,848835,2022-12-29,85245CNYS,85245CNY,85245,ECOMMERCE,35720.0,83193.0,83193,1,...,1.0,14,4,0,0,4,14,1,1,2
848935,848935,2022-12-31,85245CNYS,85245CNY,85245,ECOMMERCE,35727.0,83193.0,83193,1,...,1.0,15,0,0,1,5,15,0,0,1


In [24]:
df.columns

Index(['Unnamed: 0', 'date', 'sku', 'cc', 'itemnumber', 'warehouse', 'cost',
       'unit_price', 'sale_amount', 'quantity', 'weekday', 'month', 'year',
       'is_offer_day', 'is_metropolitan_zone', 'is_inside_mall',
       'sku_cumulative_sales_in_the_week', 'sku_cumulative_sales_in_the_month',
       'sku_cumulative_sales_in_the_year', 'sku_historic_sales',
       'sku_historic_sales_same_day_of_the_week',
       'sku_historic_sales_same_month', 'sku_warehouse_last_7days_sales',
       'sku_warehouse_last_7days_mean_sales',
       'sku_warehouse_last_10days_sales',
       'sku_warehouse_last_10days_mean_sales',
       'sku_warehouse_last_14days_sales',
       'sku_warehouse_last_14days_mean_sales',
       'sku_warehouse_last_30days_sales',
       'sku_warehouse_last_30days_mean_sales', 'sku_warehouse_historic_sales',
       'sku_warehouse_historic_sales_same_day_of_the_week',
       'sku_warehouse_historic_sales_same_month',
       'sku_warehouse_cumulative_sales_in_the_week',
     

In [25]:
# Split the data into X and y datasets
X_train = train_data.drop([
    'Unnamed: 0', 'date', 'itemnumber', 'sku', 'cc', 'warehouse',
    'cost', 'sale_amount', 'quantity', 'y_sku_warehouse_next_7days_sales',
    'y_sku_warehouse_next_14days_sales', 'y_sku_warehouse_next_30days_sales'
    ], axis=1)
y_train = train_data['y_sku_warehouse_next_7days_sales']
X_test = test_data.drop([
    'Unnamed: 0', 'date', 'itemnumber', 'sku', 'cc', 'warehouse',
    'cost', 'sale_amount', 'quantity', 'y_sku_warehouse_next_7days_sales',
    'y_sku_warehouse_next_14days_sales', 'y_sku_warehouse_next_30days_sales'
    ], axis=1)
y_test = test_data['y_sku_warehouse_next_7days_sales']

In [26]:
X_train.columns

Index(['unit_price', 'weekday', 'month', 'year', 'is_offer_day',
       'is_metropolitan_zone', 'is_inside_mall',
       'sku_cumulative_sales_in_the_week', 'sku_cumulative_sales_in_the_month',
       'sku_cumulative_sales_in_the_year', 'sku_historic_sales',
       'sku_historic_sales_same_day_of_the_week',
       'sku_historic_sales_same_month', 'sku_warehouse_last_7days_sales',
       'sku_warehouse_last_7days_mean_sales',
       'sku_warehouse_last_10days_sales',
       'sku_warehouse_last_10days_mean_sales',
       'sku_warehouse_last_14days_sales',
       'sku_warehouse_last_14days_mean_sales',
       'sku_warehouse_last_30days_sales',
       'sku_warehouse_last_30days_mean_sales', 'sku_warehouse_historic_sales',
       'sku_warehouse_historic_sales_same_day_of_the_week',
       'sku_warehouse_historic_sales_same_month',
       'sku_warehouse_cumulative_sales_in_the_week',
       'sku_warehouse_cumulative_sales_in_the_month',
       'sku_warehouse_cumulative_sales_in_the_year'],
 

In [31]:
from xgboost import XGBRegressor
regression = XGBRegressor(learning_rate=0.1)
regression.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [29]:
def check_best_params():
    from sklearn.model_selection import GridSearchCV

    model = XGBRegressor()

    param_grid = {'learning_rate': [0.1, 0.01, 0.001]}

    grid_search = GridSearchCV(model, param_grid, cv=10, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    return grid_search.best_params_

Evaluate model performance

In [32]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
y_pred = regression.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)



print("R-squared [R2]: %.2f" % r2)
print("Mean Absolute Error [MAE]: %.2f" % mae)
print("Mean Squared Error [MSE]: %.2f" % mse)
print("Root Mean Squared Error [RMSE]: %.2f" % rmse)

R-squared [R2]: -1.05
Mean Absolute Error [MAE]: 0.71
Mean Squared Error [MSE]: 3.15
Root Mean Squared Error [RMSE]: 1.78


Cross Validation

In [14]:
X = df.drop(['Unnamed: 0', 'date', 'itemnumber', 'sku', 'cc', 'warehouse', 'cost', 'sale_amount', 'quantity'], axis=1)
y = df['quantity']

In [15]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(regression, X, y, cv=5, scoring='neg_mean_squared_error')
mse_scores = -scores
rmse_scores = np.sqrt(mse_scores)
print("RMSE scores: ", rmse_scores)
print("Mean RMSE: %.2f" % rmse_scores.mean())

RMSE scores:  [1.7247756  1.55180808 1.04679638 1.52202567 2.75819716]
Mean RMSE: 1.72
